In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

4.53 ms ± 229 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## numpy random generation in jnp dot product is slow due to memory swap to GPU

In [4]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

43.7 ms ± 6.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

9.38 ms ± 488 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit np.dot(x, x.T)

35.3 ms ± 410 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

2.9 ms ± 541 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

41.5 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
def sum_logistic(x):
    return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661197 0.10499357]


In [9]:
from jax import jacfwd, jacrev
def hessian(fun):
  return jit(jacfwd(jacrev(fun)))

## Auto-vectorization w/ vmap()

In [10]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
    return jnp.dot(mat, v)

In [11]:
def naively_batched_apply_matrix(v_batched):
    return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
4.82 ms ± 4.88 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
@jit
def batched_apply_matrix(v_batched):
    return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
36.4 µs ± 761 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
@jit
def vmap_batched_apply_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
35.6 µs ± 88.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### note: vmap automatcially vectorize batched_x to perform apply_matrix!